In [50]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cairosvg
from PIL import Image
import io
import json

array = cv2.imread("/Users/muratseyhan/football-analytics/logo-color-scraping/data/logo/Ajax.png", cv2.IMREAD_UNCHANGED)
mask = array[:,:,3] != 0
opaque = array[mask]
all_bgr = opaque[:,:3]
all_rgb = all_bgr[:, [2, 1, 0]]  # swapping BGR to RGB
unique_bgr = np.unique(all_bgr, axis=0)
unique_rgb = unique_bgr[:, [2, 1, 0]]  # swapping BGR to RGB

In [44]:
# define columns and read
columns = ["color", "color_name", "hex", "R", "G", "B"]
colors_df = pd.read_csv('logo-color-scraping/data/colors.csv', names=columns, header=None)

In [45]:
def find_closest_color(rgb_color, colors_df):
    # calculate euclidean distance
    distances = np.sqrt(((colors_df[["R", "G", "B"]] - rgb_color) ** 2).sum(axis=1))
    # find the index of the smallest distance
    closest_index = distances.idxmin()
    closest_color = colors_df.iloc[closest_index]
    # return original color and closest match
    return pd.Series({"Original_R": rgb_color[0],
                      "Original_G": rgb_color[1],
                      "Original_B": rgb_color[2],
                      "Matched_Color": closest_color['color_name'],
                      "Matched_R": closest_color['R'],
                      "Matched_G": closest_color['G'],
                      "Matched_B": closest_color['B']})

# apply the function to each color in the unique_rgb array
matches_df = pd.DataFrame([find_closest_color(color, colors_df) for color in unique_rgb])
histy = pd.DataFrame(all_bgr).groupby([0,1,2])[2].count().sort_values(ascending=False).rename("cnt").reset_index().rename(columns={0:'Original_B',1:'Original_G',2:'Original_R'})
# merge the count information into matches_df
matches_df = pd.merge(matches_df, histy, on=['Original_R', 'Original_G', 'Original_B'], how='left')
matches_df = matches_df.rename(columns={'cnt': 'Original_Count'})

histy.head(20)
matches_df

,Original_R,Original_G,Original_B,Matched_Color,Matched_R,Matched_G,Matched_B,Original_Count
0,0,0,0,Black,0,0,0,7260
1,1,1,1,Black,0,0,0,81
2,2,2,2,Black,0,0,0,65
3,3,3,3,Black,0,0,0,50
4,4,4,4,Black,0,0,0,58
...,...,...,...,...,...,...,...,...
504,253,253,253,White,255,255,255,61
505,255,253,253,White,255,255,255,47
506,254,254,254,White,255,255,255,83
507,255,254,254,White,255,255,255,51


In [49]:
# group by matched RGB colors and sum the original counts
grouped_df = matches_df.groupby(['Matched_R', 'Matched_G', 'Matched_B']).agg({'Original_Count': 'sum'})

# sort the results in descending order of the sum of original counts
sorted_df = grouped_df.sort_values(by='Original_Count', ascending=False)

# reset the index
sorted_df = sorted_df.reset_index()
sorted_df

# calculate the total sum of 'cnt'
total_cnt = sorted_df['Original_Count'].sum()

# determine the threshold value
threshold = 0.05 * total_cnt

# filter the DataFrame
filtered_sorted_df = sorted_df[sorted_df['Original_Count'] > threshold]

# extract RGB values as tuples
rgb_values = [tuple(x) for x in filtered_sorted_df[["Matched_R", "Matched_G", "Matched_B"]].values]
rgb_values

[(255, 255, 255), (0, 0, 0)]

In [42]:
filtered_sorted_df

,Matched_R,Matched_G,Matched_B,Original_Count
0,252,194,0,6858
1,226,6,44,3392
2,0,79,152,3210
3,0,0,0,2913
4,165,11,94,2573


In [ ]:
# function to convert .svg to .png
def convert_svg_to_png(svg_path, png_path):
    cairosvg.svg2png(url=svg_path, write_to=png_path)

# function to convert rgb to hex
def rgb_to_hex(rgb):
    return "#{:02x}{:02x}{:02x}".format(rgb[0], rgb[1], rgb[2])

# Now, integrate with your existing JSON updating logic
with open('team_logo_data.json', 'r') as file:
    team_logo_data = json.load(file)